In [1]:
import findspark
findspark.init("/opt/manual/spark")
from pyspark.sql import SparkSession, functions as F

In [2]:
spark = (
    SparkSession.builder
    .appName("Shuffle Sort Merge Join (SMJ)")
    .master("yarn")     
    .getOrCreate())

# read order items

In [3]:
order_items = spark.read \
.option("header",True) \
.option("inferSchema", True) \
.option("sep", ",") \
.csv("/user/train/datasets/retail_db/order_items.csv")

In [4]:
order_items.count()

172198

In [5]:
order_items.limit(3).toPandas()

,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice
0,1,1,957,1,299.98,299.98
1,2,2,1073,1,199.99,199.99
2,3,2,502,5,250.00,50.00


# read orders

In [6]:
orders = spark.read \
.option("header",True) \
.option("inferSchema", True) \
.option("sep", ",") \
.csv("/user/train/datasets/retail_db/orders.csv")

In [7]:
orders.count()

68883

In [8]:
orders.limit(3).toPandas()

,orderId,orderDate,orderCustomerId,orderStatus
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE


In [13]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1") # broadcast i kapatıyoruz

# join

In [14]:
joined_df = order_items.join(orders, order_items.orderItemOrderId == orders.orderId)

In [15]:
joined_df.limit(3).toPandas()

,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice,orderId,orderDate,orderCustomerId,orderStatus
0,348,148,502,2,100.00,50.00,148,2013-07-26 00:00:00.0,5383,PROCESSING
1,349,148,502,5,250.00,50.00,148,2013-07-26 00:00:00.0,5383,PROCESSING
2,350,148,403,1,129.99,129.99,148,2013-07-26 00:00:00.0,5383,PROCESSING


In [16]:
joined_df.explain()

== Physical Plan ==
*(5) SortMergeJoin [orderItemOrderId#17], [orderId#61], Inner
:- *(2) Sort [orderItemOrderId#17 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(orderItemOrderId#17, 200), true, [id=#236]
:     +- *(1) Project [orderItemName#16, orderItemOrderId#17, orderItemProductId#18, orderItemQuantity#19, orderItemSubTotal#20, orderItemProductPrice#21]
:        +- *(1) Filter isnotnull(orderItemOrderId#17)
:           +- FileScan csv [orderItemName#16,orderItemOrderId#17,orderItemProductId#18,orderItemQuantity#19,orderItemSubTotal#20,orderItemProductPrice#21] Batched: false, DataFilters: [isnotnull(orderItemOrderId#17)], Format: CSV, Location: InMemoryFileIndex[hdfs://localhost:9000/user/train/datasets/retail_db/order_items.csv], PartitionFilters: [], PushedFilters: [IsNotNull(orderItemOrderId)], ReadSchema: struct<orderItemName:int,orderItemOrderId:int,orderItemProductId:int,orderItemQuantity:int,orderI...
+- *(4) Sort [orderId#61 ASC NULLS FIRST], false, 0
   +- Exc

In [17]:
spark.stop()